In [12]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig,RunnableGenerator
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.tools.render import render_text_description
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf
from typing import Iterator,List,AsyncIterator

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [4]:
# llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm=ChatOpenAI(model="gpt-3.5-turbo")

In [8]:
chain=RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x:x['num']*2),  # Will return a Dictionary within a dictionary
    modified=lambda x : x['num']*2  # Will return a Key-Value Pair                          
)

In [9]:
chain.invoke(input={"num":3})

{'extra': {'num': 3, 'mult': 6}, 'modified': 6}

<h3> Application in Streaming </h3>

In [14]:
vectorStore=FAISS.from_texts(
    texts=["Ritish has a 18 month old son !",
           "Rohit is Ritish's Younger Brother",
           "Eklavya is Ritish's Son"
           "Rekha is the name of Ritish's Mother",
           "Anirban is Ritish's Father"],
    embedding=OpenAIEmbeddings())

In [15]:
question={"question":"Who is Anirban's Wife and who is Eklavya's Uncle?"}

In [16]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
            HumanMessagePromptTemplate.from_template
                (
                    template=
                    """
                        Answer the question based only on the following context:
                        {context}
                
                        Question: {question}
                    """
                )
            ],
    
)

In [17]:
retrievalChain=RunnableParallel({"context":vectorStore.as_retriever(k=8),"question":RunnablePassthrough()})|RunnablePassthrough.assign(output=prompt|llm|StrOutputParser())

In [18]:
response=retrievalChain.invoke(input=question['question'])

In [19]:
response

{'context': [Document(id='e7e8db5e-aece-402c-8e4d-f4e4700e4707', metadata={}, page_content="Anirban is Ritish's Father"),
  Document(id='79e7fb34-5160-4898-8247-3b8f5b9bf10e', metadata={}, page_content="Eklavya is Ritish's SonRekha is the name of Ritish's Mother"),
  Document(id='0d6a756a-6f77-4577-99ad-dd1c552492d2', metadata={}, page_content="Rohit is Ritish's Younger Brother"),
  Document(id='c1b631fe-d7ae-43ab-8c2d-85e79f5e9753', metadata={}, page_content='Ritish has a 18 month old son !')],
 'question': "Who is Anirban's Wife and who is Eklavya's Uncle?",
 'output': "Anirban's wife is Rekha and Eklavya's uncle is Rohit."}

In [20]:
print(response['output'])

Anirban's wife is Rekha and Eklavya's uncle is Rohit.


In [21]:
for chunk in retrievalChain.stream(input=question['question']):
    print(chunk)

{'question': "Who is Anirban's Wife and who is Eklavya's Uncle?"}
{'context': [Document(id='e7e8db5e-aece-402c-8e4d-f4e4700e4707', metadata={}, page_content="Anirban is Ritish's Father"), Document(id='79e7fb34-5160-4898-8247-3b8f5b9bf10e', metadata={}, page_content="Eklavya is Ritish's SonRekha is the name of Ritish's Mother"), Document(id='0d6a756a-6f77-4577-99ad-dd1c552492d2', metadata={}, page_content="Rohit is Ritish's Younger Brother"), Document(id='c1b631fe-d7ae-43ab-8c2d-85e79f5e9753', metadata={}, page_content='Ritish has a 18 month old son !')]}
{'output': ''}
{'output': 'An'}
{'output': 'ir'}
{'output': 'ban'}
{'output': "'s"}
{'output': ' Wife'}
{'output': ' is'}
{'output': ' Re'}
{'output': 'k'}
{'output': 'ha'}
{'output': ' and'}
{'output': ' E'}
{'output': 'kl'}
{'output': 'avy'}
{'output': 'a'}
{'output': "'s"}
{'output': ' Uncle'}
{'output': ' is'}
{'output': ' Roh'}
{'output': 'it'}
{'output': '.'}
{'output': ''}
